In [82]:
from ptpl import PyTorchPipeline
from dataset import get_loaders, ds

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import random
random.seed(43)
from model import Encoder, Decoder

In [83]:
perSentence, vocab = get_loaders()
vocab_size = len(vocab)

hidden_dim = 32
emb_dim = 32
nfe = 50

hparams = {
    'wb' : False,
    'span_size': 30,
    'hidden_dim': hidden_dim,
    'emb_dim': emb_dim,
    'nfe': nfe,
    'gpu_num': 0,
    'num_epochs': 100,
    'num_batches' : 256,
    'path2load': "./weights/hidden_size_" + str(hidden_dim) + "_emb_dim_" + str(emb_dim) + "_nfe_" + str(nfe) + ".pt",
    'learning_rate': 1e-3,
}

 37%|███▋      | 49884/135842 [00:00<00:00, 498771.50it/s]

Total number of sentences is 135842 



100%|██████████| 135842/135842 [00:00<00:00, 255605.77it/s]


The vocabulary size if 5384 

The size of the newly created text data is 122885, 90% of the original text


In [84]:
device = torch.device("cuda:" + str(hparams['gpu_num']))

In [85]:
model = nn.Sequential(
    nn.Embedding(vocab_size, hparams['emb_dim']),
    Encoder(hparams['emb_dim'], hparams['hidden_dim']),
    Decoder(hparams['hidden_dim'], vocab_size, hparams['span_size'], device, hparams['nfe']),
)
model.to(device)
pass

In [86]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = hparams['learning_rate'])


# define PyTorch pipeline
ptpl = PyTorchPipeline(
    project_name = "gru_node",
    configs = {
        'device': device,
        'criterion': criterion,
        'optimizer': optimizer,
        'train_dataloader': perSentence,
        'print_logs': True,
        'wb': hparams['wb'],
    },
    hparams = hparams,
    model = model,
)

PyTorch pipeline for gru_node is set up


In [124]:
sampleSentences = random.sample(perSentence[5], k = 10)

In [126]:
for sampleSentence in sampleSentences:
    sampleSentence = " ".join([vocab.itos[w] for w in sampleSentence])
    print(sampleSentence)

you re the oldest . <EOS>
i love this store . <EOS>
whose clock is it ? <EOS>
i don t belong . <EOS>
i m from brazil . <EOS>
i was busy cooking . <EOS>
that s not interesting . <EOS>
it was no accident . <EOS>
go grab a drink . <EOS>
they looked very happy . <EOS>


In [127]:
batch = torch.cat([torch.tensor(s).view(1, -1) for s in sampleSentences])
print(batch.shape)

torch.Size([10, 6])


In [128]:
ptpl.load(hparams['path2load'])

In [129]:
output_data = ptpl.predict(batch, "train")

In [130]:
loss, selected_output_data = ptpl.compute_loss(batch, output_data)

In [131]:
print(loss)

tensor(0.0588, device='cuda:0', grad_fn=<NllLossBackward>)


In [132]:
temp = output_data.transpose(1, 0).argmax(2)
for i in range(10):
    sampleSentence = " ".join([vocab.itos[w] for w in sampleSentences[i]])
    torch_sentence = temp[i].cpu()
    torch_sentence = " ".join([vocab.itos[torch_sentence[i].item()] for i in range(torch_sentence.shape[0])])
    print(torch_sentence)
    print(sampleSentence)
    print(" ")

you you just just just re re re re re re re re re teased spoiling spoiling the the the the the the the the oldest oldest oldest oldest oldest oldest oldest oldest oldest contract contract contract . . . . . . . . . <EOS> <EOS> <EOS> <EOS>
you re the oldest . <EOS>
 
i i i i i love love love love love love love love love on on this this this this this this this this store store store store store store store store store drawer shelf . . . . . . . . . <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>
i love this store . <EOS>
 
whose whose whose whose whose whose bicycle clock clock clock clock clock clock clock umbrella is is is is is is is up s it it it it it it it it it it it it it ? ? ? ? ? ? ? ? ? <EOS> <EOS> <EOS> <EOS>
whose clock is it ? <EOS>
 
i i i just just don don don don don don don don don t t t t t t t t t speak belong belong belong belong belong belong belong belong belong jump spaghetti spaghetti spaghetti . . . . . . . . <EOS> <EOS> <EOS> <EOS> <EOS>
i don t belong . <EOS>
 
i i i ha